##### Copyright 2021 The TensorFlow Authors.

In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Migrar su código TFLite a TF2

<table class="tfo-notebook-buttons" align="left">
  <td><a target="_blank" href="https://www.tensorflow.org/guide/migrate/tflite"><img src="https://www.tensorflow.org/images/tf_logo_32px.png">Ver en TensorFlow.org</a></td>
  <td><a target="_blank" href="https://colab.research.google.com/github/tensorflow/docs-l10n/blob/master/site/es-419/guide/migrate/tflite.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png">Ejecutar en Google Colab</a></td>
  <td>     <a target="_blank" href="https://github.com/tensorflow/docs-l10n/blob/master/site/es-419/guide/migrate/tflite.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png">Ver código fuente en GitHub</a>
</td>
  <td><a href="https://storage.googleapis.com/tensorflow_docs/docs-l10n/site/es-419/guide/migrate/tflite.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png">Descargar bloc de notas</a></td>
</table>

[TensorFlow Lite](https://www.tensorflow.org/lite/guide) (TFLite) es un conjunto de herramientas que ayuda a los desarrolladores a correr inferencia ML sobre dispositivos (móviles, embebidos y dispositivos IoT). El conversor [TFLite](https://www.tensorflow.org/lite/convert) es una de esas herramientas que convierte los modelos TF existentes en un formato de modelo TFLite optimizado que puede ejecutarse de forma eficiente en el dispositivo.

En este documento, aprenderá qué cambios debe realizar en su código de conversión de TF a TFLite, seguido de algunos ejemplos que hacen lo mismo.

## Cambios en su código de conversión de TF a TFLite

- Si está usando un formato de modelo TF1 heredado (como un archivo Keras, GraphDef congelado, puntos de verificación, tf.Session), actualícelo a TF1/TF2 SavedModel y use la API del conversor TF2 `tf.lite.TFLiteConverter.from_saved_model(...)` para convertirlo en un modelo TFLite (consulte la Tabla 1).

- Actualice los indicadores de la API del convertidor (consulte la tabla 2).

- Elimine las API heredadas como `tf.lite.constants`. (por ejemplo: Reemplace `tf.lite.constants.INT8` por `tf.int8`)

// Tabla 1 // Actualización de la API del conversor Python TFLite

API TF1 | API TF2
--- | ---
`tf.lite.TFLiteConverter.from_saved_model('saved_model/',..)` | *admitida*
`tf.lite.TFLiteConverter.from_keras_model_file('model.h5',..)` | *retirada (actualización al formato SavedModel)*
`tf.lite.TFLiteConverter.from_frozen_graph('model.pb',..)` | *retirada (actualización al formato SavedModel)*
`tf.lite.TFLiteConverter.from_session(sess,...)` | *retirada (actualización al formato SavedModel)*

&lt;style&gt;   .table {margin-left: 0 !important;} &lt;/style&gt;

// Tabla 2 // Actualización de indicadores de la API del conversor TFLite Python

API TF1 | API TF2
--- | ---
`allow_custom_ops`<br>`optimizations`<br> `representative_dataset`<br>`target_spec` <br>`inference_input_type`<br>`inference_output_type`<br>`experimental_new_converter`<br> `experimental_new_quantizer` | *admitidas* <br><br><br><br><br><br><br><br>
`input_tensors`<br>`output_tensors`<br>`input_arrays_with_shape`<br>`output_arrays`<br>`experimental_debug_info_func` | *retiradas (argumentos de la API del convertidor no admitidos)*<br><br><br><br><br>
`change_concat_input_ranges`<br>`default_ranges_stats`<br>`get_input_arrays()`<br>`inference_type`<br>`quantized_input_stats`<br> `reorder_across_fake_quant` | *retiradas (flujos de trabajo de cuantización no admitidos)*<br><br><br><br><br><br>
`conversion_summary_dir`<br>`dump_graphviz_dir`<br>`dump_graphviz_video` | *retiradas (en su lugar, visualice los modelos usando [Netron](https://lutzroeder.github.io/netron/) o [visualize.py](https://github.com/tensorflow/tensorflow/blob/master/tensorflow/lite/tools/visualize.py))*<br><br><br>
`output_format`<br>`drop_control_dependency` | *retiradas (características no soportadas en TF2)*<br><br>

## Ejemplos

Ahora verá algunos ejemplos para convertir modelos heredados de TF1 en TF1/TF2 SavedModels y luego convertirlos en modelos TF2 TFLite.

### Preparación

Comience con las importaciones necesarias de TensorFlow.

In [ ]:
import tensorflow as tf
import tensorflow.compat.v1 as tf1
import numpy as np

import logging
logger = tf.get_logger()
logger.setLevel(logging.ERROR)

import shutil
def remove_dir(path):
  try:
    shutil.rmtree(path)
  except:
    pass

Cree todos los formatos de modelo TF1 necesarios.

In [ ]:
# Create a TF1 SavedModel
SAVED_MODEL_DIR = "tf_saved_model/"
remove_dir(SAVED_MODEL_DIR)
with tf1.Graph().as_default() as g:
  with tf1.Session() as sess:
    input = tf1.placeholder(tf.float32, shape=(3,), name='input')
    output = input + 2
    # print("result: ", sess.run(output, {input: [0., 2., 4.]}))
    tf1.saved_model.simple_save(
        sess, SAVED_MODEL_DIR,
        inputs={'input': input}, 
        outputs={'output': output})
print("TF1 SavedModel path: ", SAVED_MODEL_DIR)

# Create a TF1 Keras model
KERAS_MODEL_PATH = 'tf_keras_model.h5'
model = tf1.keras.models.Sequential([
    tf1.keras.layers.InputLayer(input_shape=(128, 128, 3,), name='input'),
    tf1.keras.layers.Dense(units=16, input_shape=(128, 128, 3,), activation='relu'),
    tf1.keras.layers.Dense(units=1, name='output')
])
model.save(KERAS_MODEL_PATH, save_format='h5')
print("TF1 Keras Model path: ", KERAS_MODEL_PATH)

# Create a TF1 frozen GraphDef model
GRAPH_DEF_MODEL_PATH = tf.keras.utils.get_file(
    'mobilenet_v1_0.25_128',
    origin='https://storage.googleapis.com/download.tensorflow.org/models/mobilenet_v1_0.25_128_frozen.tgz',
    untar=True,
) + '/frozen_graph.pb'

print("TF1 frozen GraphDef path: ", GRAPH_DEF_MODEL_PATH)

### 1. Convierta un TF1 SavedModel en un modelo TFLite


#### Antes: Conversión con TF1

Este es el código típico para la conversión TFlite estilo TF1.


In [ ]:
converter = tf1.lite.TFLiteConverter.from_saved_model(
    saved_model_dir=SAVED_MODEL_DIR,
    input_arrays=['input'],
    input_shapes={'input' : [3]}
)
converter.optimizations = {tf.lite.Optimize.DEFAULT}
converter.change_concat_input_ranges = True
tflite_model = converter.convert()
# Ignore warning: "Use '@tf.function' or '@defun' to decorate the function."

#### Después: Conversión con TF2

Convierte directamente el TF1 SavedModel en un modelo TFLite, estableciendo unos indicadores de conversión v2 más pequeños.

In [ ]:
# Convert TF1 SavedModel to a TFLite model.
converter = tf.lite.TFLiteConverter.from_saved_model(saved_model_dir=SAVED_MODEL_DIR)
converter.optimizations = {tf.lite.Optimize.DEFAULT}
tflite_model = converter.convert()

### 2. Convierta un archivo de modelo Keras TF1 en un modelo TFLite

#### Antes: Conversión con TF1

Este es el código típico para la conversión TFlite estilo TF1.

In [ ]:
converter = tf1.lite.TFLiteConverter.from_keras_model_file(model_file=KERAS_MODEL_PATH)
converter.optimizations = {tf.lite.Optimize.DEFAULT}
converter.change_concat_input_ranges = True
tflite_model = converter.convert()

#### Después: Conversión con TF2

Primero, convierta el archivo del modelo Keras TF1 en un SavedModel TF2 y, a continuación, conviértalo en un modelo TFLite, estableciendo unos indicadores de conversión v2 más pequeños.

In [ ]:
# Convert TF1 Keras model file to TF2 SavedModel.
model = tf.keras.models.load_model(KERAS_MODEL_PATH)
model.save(filepath='saved_model_2/')

# Convert TF2 SavedModel to a TFLite model.
converter = tf.lite.TFLiteConverter.from_saved_model(saved_model_dir='saved_model_2/')
tflite_model = converter.convert()

### 3. Convertir un GraphDef congelado TF1 en un modelo TFLite


#### Antes: Conversión con TF1

Este es el código típico para la conversión TFlite estilo TF1.

In [ ]:
converter = tf1.lite.TFLiteConverter.from_frozen_graph(
    graph_def_file=GRAPH_DEF_MODEL_PATH,
    input_arrays=['input'],
    input_shapes={'input' : [1, 128, 128, 3]},
    output_arrays=['MobilenetV1/Predictions/Softmax'],
)
converter.optimizations = {tf.lite.Optimize.DEFAULT}
converter.change_concat_input_ranges = True
tflite_model = converter.convert()

#### Después: Conversión con TF2

Primero, convierta el GraphDef congelado TF1 en un SavedModel TF1 y, a continuación, conviértalo en un modelo TFLite, con unos indicadores de convertidor v2 más pequeños configurados.


In [ ]:
## Convert TF1 frozen Graph to TF1 SavedModel.

# Load the graph as a v1.GraphDef
import pathlib
gdef = tf.compat.v1.GraphDef()
gdef.ParseFromString(pathlib.Path(GRAPH_DEF_MODEL_PATH).read_bytes())

# Convert the GraphDef to a tf.Graph
with tf.Graph().as_default() as g:
  tf.graph_util.import_graph_def(gdef, name="")

# Look up the input and output tensors.
input_tensor = g.get_tensor_by_name('input:0') 
output_tensor = g.get_tensor_by_name('MobilenetV1/Predictions/Softmax:0')

# Save the graph as a TF1 Savedmodel
remove_dir('saved_model_3/')
with tf.compat.v1.Session(graph=g) as s:
  tf.compat.v1.saved_model.simple_save(
      session=s,
      export_dir='saved_model_3/',
      inputs={'input':input_tensor},
      outputs={'output':output_tensor})

# Convert TF1 SavedModel to a TFLite model.
converter = tf.lite.TFLiteConverter.from_saved_model(saved_model_dir='saved_model_3/')
converter.optimizations = {tf.lite.Optimize.DEFAULT}
tflite_model = converter.convert()

# Lecturas adicionales

- Consulte la [Guía de TFLite](https://www.tensorflow.org/lite/guide) para saber más sobre los flujos de trabajo y las funciones más recientes.
- Si está usando código TF1 o formatos de modelos TF1 heredados (archivos Keras `.h5`, GraphDef `.pb` congelados, etc.), actualice su código y migre sus modelos al formato [TF2 SavedModel](https://www.tensorflow.org/guide/saved_model).
